In [ ]:
from os import path,chdir
import sys
chdir(path.dirname(path.realpath(sys.argv[0]))) # change working directory to script location

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from scripts.improve_result import improve_result
from huggingface_hub import login

In [ ]:
login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Tony177/codellama-13b-dockerfile-generation")
model = AutoModelForCausalLM.from_pretrained("Tony177/codellama-13b-dockerfile-generation")
model.load_adapter("./codellama-13b-dockerfile-generation/PEFT_CUSTOM")


In [ ]:
def generate_text(tokenizer, model, prompt: str) -> str:
    prompt += ". Output must be in valid Dockerfile format. Output must include only dockerfile commands."
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda") # Added last part to avoid crash to KeyError: 'shape'
    gen_tokens = model.generate(input_ids, max_new_tokens=512, num_beams=5, no_repeat_ngram_size=2, early_stopping=False,pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0] # One element list, just the response
    return improve_result(prompt,result)

In [ ]:
def generate_text_INST(tokenizer, model, prompt: str) -> str:
    prompt += ". Output must be in valid Dockerfile format. Output must include only dockerfile commands. [/INST]"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda") # Added last part to avoid crash to KeyError: 'shape'
    gen_tokens = model.generate(input_ids, max_new_tokens=512, num_beams=5, no_repeat_ngram_size=2, early_stopping=False,pad_token_id=tokenizer.eos_token_id)
    result = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0] # One element list, just the response
    return improve_result(prompt,result)

In [ ]:
generate_text(tokenizer, model, "Generate a dockerfile of Wordpress 5.7")

In [ ]:
generate_text_INST(tokenizer, model, "<s>[INST] Generate a dockerfile of Wordpress 5.7")

In [ ]:
generate_text(tokenizer, model, "Generate a dockerfile of Python 3.7")

In [ ]:
generate_text_INST(tokenizer, model, "<s>[INST] Generate a dockerfile of Python 3.7")

In [ ]:
generate_text(tokenizer, model, "Generate a Dockerfile of Ruby 3.2.1")

In [ ]:
generate_text_INST(tokenizer, model, "<s>[INST] Generate a Dockerfile of Ruby 3.2.1")